In [111]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from matplotlib import colors
from typing import Dict, List, Optional, Text, Tuple

In [112]:
INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph',
                  'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']
OUTPUT_FEATURES = ['FireMask']
#file_pattern = '/replace/with/you/own/path/to/tfrecords'
file_pattern = '/Users/kypark/work/task/Disaster_Detection/Wildfire_Spread/dataset/next_day_wildfire_spread_train*'

In [113]:
def _get_features_desc(
    sample_size: int,
    features: List[Text],
) -> Dict[Text, tf.io.FixedLenFeature]:
  """Creates a features dictionary for TensorFlow IO.
  Args:
    sample_size: Size of the input tiles (square).
    features: List of feature names.
  Returns:
    A features dictionary for TensorFlow IO.
  """
  sample_shape = [sample_size, sample_size]
  features = set(features)
  columns = [
      tf.io.FixedLenFeature(shape=sample_shape, dtype=tf.float32)
      for _ in features
  ]
  return dict(zip(features, columns))


def _parse_function(example_proto: tf.train.Example, data_size: int):
  features_desc = _get_features_desc(data_size, INPUT_FEATURES + OUTPUT_FEATURES)
  features_dict = tf.io.parse_single_example(example_proto, features_desc)  # {feat_name: feat_ndarray, ...}
  return features_dict

In [114]:
dataset1 = tf.data.Dataset.list_files(file_pattern)
print('type of dataset: {}'.format(type(dataset1)))

type of dataset: <class 'tensorflow.python.data.ops.dataset_ops.ShuffleDataset'>


In [115]:
dataset2 = dataset1.interleave(\
  lambda x: tf.data.TFRecordDataset(x, compression_type=None),\
  num_parallel_calls=tf.data.experimental.AUTOTUNE)
print('type of dataset: {}'.format(type(dataset2)))

type of dataset: <class 'tensorflow.python.data.ops.dataset_ops.ParallelInterleaveDataset'>


In [116]:
dataset3 = dataset2.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print('type of dataset: {}'.format(type(dataset3)))

type of dataset: <class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [117]:
dataset4 = dataset3.map(\
  lambda x: _parse_function(x, 64),\
  num_parallel_calls=tf.data.experimental.AUTOTUNE)
print('type of dataset: {}'.format(type(dataset4)))

type of dataset: <class 'tensorflow.python.data.ops.dataset_ops.ParallelMapDataset'>


In [118]:
train_cases = list(dataset4.as_numpy_iterator())
num_train_cases = len(train_cases)
feat_names = sorted(train_cases[0].keys())
print('* total # of training cases: {}'.format(num_train_cases))
print('* type of each train case: {}'.format(type(train_cases[0])))
print('* feature names in sorted order: {}'.format(feat_names))
print('* feature config:')
print('------------------------------------------------------------')
print('{:>20s}{:>20s}{:>20s}'.format('feature name', 'feature shape', 'dtype'))
print('------------------------------------------------------------')
for k, v in train_cases[0].items():
    print('{:>20s}{:>20s}{:>20s}'.format(k, str(v.shape), str(v.dtype)))
print('------------------------------------------------------------')

* total # of training cases: 14979
* type of each train case: <class 'dict'>
* feature names in sorted order: ['FireMask', 'NDVI', 'PrevFireMask', 'elevation', 'erc', 'pdsi', 'population', 'pr', 'sph', 'th', 'tmmn', 'tmmx', 'vs']
* feature config:
------------------------------------------------------------
        feature name       feature shape               dtype
------------------------------------------------------------
            FireMask            (64, 64)             float32
                NDVI            (64, 64)             float32
        PrevFireMask            (64, 64)             float32
           elevation            (64, 64)             float32
                 erc            (64, 64)             float32
                pdsi            (64, 64)             float32
          population            (64, 64)             float32
                  pr            (64, 64)             float32
                 sph            (64, 64)             float32
                  th

In [119]:
# Data statistics (from the original paper's repo)
# For each variable, the statistics are ordered in the form:
# (min_clip, max_clip, mean, std)
DATA_STATS = {
    # 0.1 percentile, 99.9 percentile
    'elevation': (0.0, 3141.0, 657.3003, 649.0147),
    # Pressure
    # 0.1 percentile, 99.9 percentile
    'pdsi': (-6.1298, 7.8760, -0.0053, 2.6823),
    'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677),
    # Precipitation in mm.
    # Negative values make no sense, so min is set to 0.
    # 0., 99.9 percentile
    'pr': (0.0, 44.5304, 1.7398051, 4.4828),
    # Specific humidity ranges from 0 to 100%.
    'sph': (0., 1., 0.0071658953, 0.0042835088),
    # Wind direction in degrees clockwise from north.
    # Thus min set to 0 and max set to 360.
    'th': (0., 360.0, 190.3298, 72.5985),
    # Min/max temperature in Kelvin.
    # -20 degree C, 99.9 percentile
    'tmmn': (253.15, 298.9489, 281.08768, 8.9824),
    # -20 degree C, 99.9 percentile
    'tmmx': (253.15, 315.0923, 295.17383, 9.8155),
    # Wind speed.
    # Negative values do not make sense, given there is a wind direction.
    # 0., 99.9 percentile
    'vs': (0.0, 10.0243, 3.8501, 1.4110),
    # NFDRS fire danger index energy release component expressed in BTU's per
    # square foot.
    # Negative values do not make sense. Thus min set to zero.
    # 0., 99.9 percentile
    'erc': (0.0, 106.2489, 37.3263, 20.8460),
    # Population
    # min, 99.9 percentile
    'population': (0., 2534.0630, 25.5314, 154.7233),
    # We don't want to normalize the FireMasks.
    'PrevFireMask': (-1., 1., 0., 1.),
    'FireMask': (-1., 1., 0., 1.)
}

In [120]:
log_interval = 100
numeric_feat_names = [feat_name for feat_name in feat_names
                      if feat_name not in ['PrevFireMask', 'FireMask']]
feat_val = {feat_name: None for feat_name in numeric_feat_names}
feat_min = {feat_name: 0.0 for feat_name in numeric_feat_names}
feat_max = {feat_name: 0.0 for feat_name in numeric_feat_names}
feat_mean = {feat_name: 0.0 for feat_name in numeric_feat_names}
feat_std = {feat_name: 0.0 for feat_name in numeric_feat_names}
for feat_name in numeric_feat_names:
    feat_val[feat_name] = []
    for idx, train_case in enumerate(train_cases):
        feat_val[feat_name].extend(train_case[feat_name].reshape(-1))
#         if (idx + 1) % log_interval == 0:
#             print('\tcase idx: {}'.format(idx + 1))
#     print('\tcase idx: {}'.format(idx + 1))
    print('[{}] len: {} ================='.format(feat_name, len(feat_val[feat_name])))
    feat_min[feat_name] = np.min(feat_val[feat_name])
    feat_max[feat_name] = np.max(feat_val[feat_name])
    feat_mean[feat_name] = np.mean(feat_val[feat_name])
    feat_std[feat_name] = np.std(feat_val[feat_name])
    print('\tmin: {}, max: {}, mean: {}, std: {}'.format(
        feat_min[feat_name], feat_max[feat_name],
        feat_mean[feat_name], feat_std[feat_name]))
#     break

[NDVI] len: 61353984 =================
	min: -9567.0, max: 9966.0, mean: 5350.6806640625, std: 2185.218994140625
[elevation] len: 61353984 =================
	min: -45.0, max: 4193.0, mean: 896.5714721679688, std: 842.6110229492188
[erc] len: 61353984 =================
	min: -1196.088623046875, max: 2470.88232421875, mean: 53.469173431396484, std: 25.09796905517578
[pdsi] len: 61353984 =================
	min: -125.71087646484375, max: 52.26896667480469, mean: -0.7728708982467651, std: 2.4407196044921875
[population] len: 61353984 =================
	min: 0.0, max: 27103.60546875, mean: 30.460250854492188, std: 214.20028686523438
[pr] len: 61353984 =================
	min: -167.44830322265625, max: 56.21477127075195, mean: 0.323428750038147, std: 1.5336633920669556
[sph] len: 61353984 =================
	min: -0.1289878934621811, max: 0.08553028106689453, mean: 0.006526323966681957, std: 0.0037355388049036264
[th] len: 61353984 =================
	min: -505870.0625, max: 37735.62890625, mean

In [121]:
# TODO try with scipy.stats.norm.ppf

# {feat_name: [min_clip, max_clip], ...}
my_clips = {feat_name: [0.0, 0.0] for feat_name in numeric_feat_names}

# recalculate min_clip/max_clip according to the original authors' logic
# as the dataset seems to be different from that of the authors' paper

# elevation: 0.1 percentile, 99.9 percentile
my_clips['elevation'][0] = np.percentile(feat_val['elevation'], 0.001)
my_clips['elevation'][1] = np.percentile(feat_val['elevation'], 0.999)
# pdsi: 0.1 percentile, 99.9 percentile
my_clips['pdsi'][0] = np.percentile(feat_val['pdsi'], 0.001)
my_clips['pdsi'][1] = np.percentile(feat_val['pdsi'], 0.999)
# NDVI: NA(0.1 percentile, 99.9 percentile by default)
my_clips['NDVI'][0] = np.percentile(feat_val['NDVI'], 0.001)
my_clips['NDVI'][1] = np.percentile(feat_val['NDVI'], 0.999)
# pr: Precipitation in mm. Negative values make no sense, so min is set to 0. 0., 99.9 percentile
my_clips['pr'][0] = 0.0
my_clips['pr'][1] = np.percentile(feat_val['pr'], 0.999)
# sph: Specific humidity ranges from 0 to 100%.
my_clips['sph'][0] = 0.0
my_clips['sph'][1] = 1.0
# th: Wind direction in degrees clockwise from north. Thus min set to 0 and max set to 360.
my_clips['th'][0] = 0.0
my_clips['th'][1] = 360.0
# tmmn: min temperature in Kelvin. -20 degree C, 99.9 percentile
my_clips['tmmn'][0] = 253.15
my_clips['tmmn'][1] = np.percentile(feat_val['tmmn'], 0.999)
# tmmx: max temperature in Kelvin. -20 degree C, 99.9 percentile
my_clips['tmmx'][0] = 253.15
my_clips['tmmx'][1] = np.percentile(feat_val['tmmx'], 0.999)
# vs: Wind speed. Negative values do not make sense, given there is a wind direction. 0., 99.9 percentile
my_clips['vs'][0] = 0.0
my_clips['vs'][1] = np.percentile(feat_val['vs'], 0.999)
# erc: NFDRS fire danger index energy release component expressed in BTU's per square foot.
#      Negative values do not make sense. Thus min set to zero. 0., 99.9 percentile
my_clips['erc'][0] = 0.0
my_clips['erc'][1] = np.percentile(feat_val['erc'], 0.999)
# population: Population. 0., 99.9 percentile
my_clips['population'][0] = 0.0
my_clips['population'][1] = np.percentile(feat_val['population'], 0.999)

In [125]:
for feat_name in my_clips:
    print('[{}] min_clip: {}, max_clip: {}'.format(feat_name, my_clips[feat_name][0], my_clips[feat_name][1]))
non_zero_pop = 0
for pop in feat_val['population']:
    if pop > 0:
        non_zero_pop += 1
print('non_zero_pop: {}'.format(non_zero_pop))

[NDVI] min_clip: -7969.0, max_clip: 0.0
[elevation] min_clip: -14.0, max_clip: 2.0
[erc] min_clip: 0.0, max_clip: 11.26390417131424
[pdsi] min_clip: -7.949708461761475, max_clip: -5.144203186035156
[population] min_clip: 0.0, max_clip: 0.0
[pr] min_clip: 0.0, max_clip: -0.022553352639079094
[sph] min_clip: 0.0, max_clip: 1.0
[th] min_clip: 0.0, max_clip: 360.0
[tmmn] min_clip: 253.15, max_clip: 264.7126858572388
[tmmx] min_clip: 253.15, max_clip: 277.5990383963013
[vs] min_clip: 0.0, max_clip: 1.3849763024055959
non_zero_pop: 39773098


In [122]:
# {feat_name: [mean, std], ...}
mean_stds = {feat_name: [0.0, 0.0] for feat_name in numeric_feat_names}
head = '{:<15s}{:<5s}{:>20s}{:>20s}{:>20s}'.format('Feature Name', 'Stat', 'Original',
                                                   'Mine', 'Original/Mine')
print(head)
print('-' * len(head))
for feat_name in mean_stds:
    a = [val if val >= my_clips[feat_name][0] else my_clips[feat_name][0]
         for val in feat_val[feat_name]]
    a = [val if val <= my_clips[feat_name][1] else my_clips[feat_name][1]
         for val in a]
    mean_stds[feat_name][0] = np.mean(a)
    mean_stds[feat_name][1] = np.std(a)
    print('{:<15s}mean {:>20.4f}{:>20.4f}{:>20.4f}'.format(feat_name, DATA_STATS[feat_name][2],
        mean_stds[feat_name][0], DATA_STATS[feat_name][2]/mean_stds[feat_name][0]))
    print('{:<15s}std  {:>20.4f}{:>20.4f}{:>20.4f}'.format('', DATA_STATS[feat_name][3],
        mean_stds[feat_name][1], DATA_STATS[feat_name][3]/mean_stds[feat_name][1]))

Feature Name   Stat             Original                Mine       Original/Mine
--------------------------------------------------------------------------------
NDVI           mean            5157.6250            -17.1077           -301.4802
               std             2466.6677            229.1324             10.7653
elevation      mean             657.3003              1.9806            331.8696
               std              649.0147              0.2086           3111.9711
erc            mean              37.3263             11.1931              3.3348
               std               20.8460              0.8242             25.2923
pdsi           mean              -0.0053             -5.1485              0.0010
               std                2.6823              0.0576             46.5946


/var/folders/2r/3ppd73zj1h90q3k5yy1gxkf40000gn/T/ipykernel_43406/34622936.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  mean_stds[feat_name][0], DATA_STATS[feat_name][2]/mean_stds[feat_name][0]))
/var/folders/2r/3ppd73zj1h90q3k5yy1gxkf40000gn/T/ipykernel_43406/34622936.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  mean_stds[feat_name][1], DATA_STATS[feat_name][3]/mean_stds[feat_name][1]))


population     mean              25.5314              0.0000                 inf
               std              154.7233              0.0000                 inf
pr             mean               1.7398             -0.0226            -77.1418
               std                4.4828              0.0000                 inf
sph            mean               0.0072              0.0065              1.0980
               std                0.0043              0.0037              1.1467
th             mean             190.3298            199.4255              0.9544
               std               72.5985             71.5846              1.0142
tmmn           mean             281.0877            264.6543              1.0621
               std                8.9824              0.7447             12.0610
tmmx           mean             295.1738            277.4927              1.0637
               std                9.8155              1.4984              6.5508
vs             mean         